In [25]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from lib.residental_data_source import ResidentialDataSource
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [26]:
data_source = ResidentialDataSource('src/dataset_prepared.xlsx')
ds = data_source.fetch_data_set()

## Standardization



### int, float

In [27]:
def standardize_numeric_data(ds, target_column='Таргет'):

    numerical_data = ds.select_dtypes(include=['int64', 'float64']).drop(columns=[target_column])

    std = StandardScaler()
    std_data = std.fit_transform(numerical_data)
    cols = std.get_feature_names_out()
    
    std_ds = pd.DataFrame(std_data, columns=cols)
    ds = pd.concat([ds.drop(columns=cols), std_ds], axis=1)
    
    return ds


In [28]:
ds = standardize_numeric_data(ds, target_column='Таргет')
ds.head()

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Всероссийский застройщик,Район,Инфраструктура: Рядом ВУЗ,Есть железная дорога 250м,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,Артем,СЗВОСХОД,Типовой,Строится,Август 2021,95000.000000,Нет,Артем,Да,Нет,...,4.070102,4.25331,4.028442,3.883871,4.281629,3.979713,4.216895,3.58124,-3.524579,-1.834763
1,Артем,СЗВОСХОД,Типовой,Строится,Ноябрь 2021,105683.431953,Нет,Артем,Да,Нет,...,4.070102,4.25331,4.028442,3.883871,4.281629,3.979713,4.216895,3.58124,-3.524579,-1.834763
2,Артем,СЗВОСХОД,Типовой,Строится,Декабрь 2021,110317.929760,Нет,Артем,Да,Нет,...,4.070102,4.25331,4.028442,3.883871,4.281629,3.979713,4.216895,3.58124,-3.524579,-1.834763
3,Артем,СЗВОСХОД,Типовой,Строится,Январь 2022,115389.053254,Нет,Артем,Да,Нет,...,4.070102,4.25331,4.028442,3.883871,4.281629,3.979713,4.216895,3.58124,-3.524579,-1.834763
4,Артем,СЗВОСХОД,Типовой,Строится,Февраль 2022,132843.750000,Нет,Артем,Да,Нет,...,4.070102,4.25331,4.028442,3.883871,4.281629,3.979713,4.216895,3.58124,-3.524579,-1.834763


### object


In [29]:
ids = (ds.dtypes == object)
object_columns = ds.columns[ids]
object_columns

Index(['Населенный пункт', 'Группа компаний', 'Класс недвижимости', 'Статус',
       'Месяц', 'Всероссийский застройщик', 'Район',
       'Инфраструктура: Рядом ВУЗ', 'Есть железная дорога 250м',
       'Есть ЛЭП 100м', 'Есть магистральные дороги 100м', 'Есть АЗС 100м',
       'Есть скверы 500м', 'Есть лес 500м', 'Есть водоем 500м',
       'Есть пляж 1000м', 'Есть тюрьма 250м', 'Есть ПНД 250м',
       'Есть промзона 250м', 'Есть гаражи 500м', 'Станции электропоезда 1000м',
       'Уехать без пересадок: Луговая', 'Уехать без пересадок: Центр',
       'Уехать без пересадок: Первая речка',
       'Уехать без пересадок: Вторая речка',
       'Уехать без пересадок: Третья рабочая',
       'Уехать без пересадок: Окатовая', 'Уехать без пересадок: Баляева',
       'Уехать без пересадок: Кампус ДВФУ', 'Основной материал фасада',
       'Вспомогательный материал фасада', 'Тип дома', 'Тип фасада',
       'Есть квартиры без отделки', 'Есть квартиры с черновой отделкой',
       'Есть квартиры с чис

### Обработка месяца

In [30]:
ds['Месяц']

0        Август 2021
1        Ноябрь 2021
2       Декабрь 2021
3        Январь 2022
4       Февраль 2022
            ...     
1021        Май 2022
1022       Июнь 2022
1023       Июль 2022
1024     Август 2022
1025     Август 2022
Name: Месяц, Length: 1026, dtype: object

In [31]:
months_mapping = {
    'январь': '1', 'февраль': '2', 'март': '3', 'апрель': '4', 'май': '5', 'июнь': '6',
    'июль': '7', 'август': '8', 'сентябрь': '9', 'октябрь': '10', 'ноябрь': '11', 'декабрь': '12'
}

month = ds['Месяц'].str.split().str[0].str.lower()
ds['Месяц'] = month.map(months_mapping)
ds['Месяц']

0        8
1       11
2       12
3        1
4        2
        ..
1021     5
1022     6
1023     7
1024     8
1025     8
Name: Месяц, Length: 1026, dtype: object

Обработка регистра и уникальных значений

In [32]:
object_list = [
    'Инфраструктура: Рядом ВУЗ', 
    'Есть железная дорога 250м',
    'Есть ЛЭП 100м',
    'Есть магистральные дороги 100м',
    'Есть АЗС 100м',
    'Есть скверы 500м',
    'Есть лес 500м',
    'Есть водоем 500м',
    'Есть пляж 1000м',
    'Есть тюрьма 250м',
    'Есть ПНД 250м',
    'Есть промзона 250м',
    'Есть гаражи 500м',
    'Есть велопарковка',
    'Есть квартиры с чистовой отделкой',
    'Есть площадки для собак', 
    'Есть квартиры под ключ', 
    'Есть квартиры без отделки', 
    'Есть квартиры с черновой отделкой', 
    'Уехать без пересадок: Луговая',
    'Уехать без пересадок: Центр',
    'Уехать без пересадок: Первая речка',
    'Уехать без пересадок: Вторая речка',
    'Уехать без пересадок: Третья рабочая',
    'Уехать без пересадок: Окатовая',
    'Уехать без пересадок: Баляева',
    'Уехать без пересадок: Кампус ДВФУ',
    'Внутренний двор без машин', 
    'Видеонаблюдение во дворе', 
    'Закрытая территория', 
    'Инфраструктура: Рядом ВУЗ', 
    'Всероссийский застройщик', 
    'Станции электропоезда 1000м']

In [33]:
for col in object_columns:
    try:
        ds[col] = ds[col].str.lower().str.strip()
    except Exception as e:
        print(f"Ошибка в столбце '{col}': {e}")

In [34]:
for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")

Column 'Есть велопарковка' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры с чистовой отделкой' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть площадки для собак' has 3 unique values: ['нет' 'неизвестно' 'да']
Column 'Есть квартиры под ключ' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры без отделки' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Есть квартиры с черновой отделкой' has 3 unique values: ['да' 'неизвестно' 'нет']
Column 'Внутренний двор без машин' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Видеонаблюдение во дворе' has 3 unique values: ['неизвестно' 'да' 'нет']
Column 'Закрытая территория' has 3 unique values: ['да' 'нет' 'неизвестно']


In [35]:
def preprocess_categorical_columns(ds, object_columns):
    for col in object_columns:
        if ds[col].dtype == 'object' and 'неизвестно' in ds[col].unique():
            target_yes = ds.loc[ds[col] == 'да', 'Таргет']
            target_no = ds.loc[ds[col] == 'нет', 'Таргет']
            median_yes = target_yes.median()
            median_no = target_no.median()
            unknown_indices = ds[ds[col] == 'неизвестно'].index

            for ind in unknown_indices:
                target_value = ds.loc[ind, 'Таргет']
                dist_yes = abs(target_value - median_yes)
                dist_no = abs(target_value - median_no)
                
                if dist_yes < dist_no:
                    ds.loc[ind, col] = 'да'
                else:
                    ds.loc[ind, col] = 'нет'
                    
    return ds


preprocess_categorical_columns(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Всероссийский застройщик,Район,Инфраструктура: Рядом ВУЗ,Есть железная дорога 250м,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,артем,сзвосход,типовой,строится,8,95000.000000,нет,артем,да,нет,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
1,артем,сзвосход,типовой,строится,11,105683.431953,нет,артем,да,нет,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
2,артем,сзвосход,типовой,строится,12,110317.929760,нет,артем,да,нет,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
3,артем,сзвосход,типовой,строится,1,115389.053254,нет,артем,да,нет,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
4,артем,сзвосход,типовой,строится,2,132843.750000,нет,артем,да,нет,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,вольно-надеждинское,сз днс дом,комфорт,строится,5,151497.433558,нет,дальний пригород,нет,нет,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1022,вольно-надеждинское,сз днс дом,комфорт,строится,6,129552.143940,нет,дальний пригород,нет,нет,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1023,вольно-надеждинское,сз днс дом,комфорт,строится,7,103214.146868,нет,дальний пригород,нет,нет,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1024,вольно-надеждинское,сз днс дом,комфорт,строится,8,110000.000000,нет,дальний пригород,нет,нет,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537


In [36]:
# checking unique values if there are more then 2

for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")

In [37]:
def standardize_categorical_data(ds, object_columns):
    
    for col in object_columns:
        ds.loc[ds[col] == 'нет', col] = 0
        ds.loc[ds[col] == 'да', col] = 1
        ds[col] = ds[col].astype(int)
    
    return ds


In [38]:
standardize_categorical_data(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,Всероссийский застройщик,Район,Инфраструктура: Рядом ВУЗ,Есть железная дорога 250м,...,Расстояние по навигатору до: Луговая,Расстояние по навигатору до: Центр,Расстояние по навигатору до: Первая речка,Расстояние по навигатору до: Вторая речка,Расстояние по навигатору до: Третья рабочая,Расстояние по навигатору до: Окатовая,Расстояние по навигатору до: Баляева,Расстояние по навигатору до: Кампус ДВФУ,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское
0,артем,сзвосход,типовой,строится,8,95000.000000,0,артем,1,0,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
1,артем,сзвосход,типовой,строится,11,105683.431953,0,артем,1,0,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
2,артем,сзвосход,типовой,строится,12,110317.929760,0,артем,1,0,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
3,артем,сзвосход,типовой,строится,1,115389.053254,0,артем,1,0,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
4,артем,сзвосход,типовой,строится,2,132843.750000,0,артем,1,0,...,4.070102,4.253310,4.028442,3.883871,4.281629,3.979713,4.216895,3.581240,-3.524579,-1.834763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,вольно-надеждинское,сз днс дом,комфорт,строится,5,151497.433558,0,дальний пригород,0,0,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1022,вольно-надеждинское,сз днс дом,комфорт,строится,6,129552.143940,0,дальний пригород,0,0,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1023,вольно-надеждинское,сз днс дом,комфорт,строится,7,103214.146868,0,дальний пригород,0,0,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537
1024,вольно-надеждинское,сз днс дом,комфорт,строится,8,110000.000000,0,дальний пригород,0,0,...,2.229393,1.923662,2.095603,1.702542,2.021271,1.859440,1.975610,2.095373,-1.745645,-3.047537


### object: multicategorical data 

In [39]:
object_ds = ds.select_dtypes(include=['object'])
object_columns = object_ds.columns

multi_categorical_cols = list(set(object_ds.columns.tolist()) - set(object_list))
object_ds[multi_categorical_cols].head()


,Вспомогательный материал фасада,Населенный пункт,Группа компаний,Основной материал фасада,Класс недвижимости,Район,Тип фасада,Месяц,Тип дома,Статус
0,стекло,артем,сзвосход,камень,типовой,артем,панель,8,монолит,строится
1,стекло,артем,сзвосход,камень,типовой,артем,панель,11,монолит,строится
2,стекло,артем,сзвосход,камень,типовой,артем,панель,12,монолит,строится
3,стекло,артем,сзвосход,камень,типовой,артем,панель,1,монолит,строится
4,стекло,артем,сзвосход,камень,типовой,артем,панель,2,монолит,строится


In [40]:
def encode_categorical_features(ds, cols):

    ohe = OneHotEncoder()
    ohe.fit(ds[cols])
    ohe_data = ohe.transform(ds[cols]).toarray()
    ohe_cols = ohe.get_feature_names_out()
    ohe_df = pd.DataFrame(ohe_data, columns=ohe_cols)

    ds = pd.concat([ds.drop(cols, axis=1), ohe_df], axis=1)
    return ds

encoded_object_ds = encode_categorical_features(object_ds, multi_categorical_cols)
encoded_object_ds.head()


,Вспомогательный материал фасада_дерево,Вспомогательный материал фасада_другой,Вспомогательный материал фасада_керамогранит,Вспомогательный материал фасада_кирпич,Вспомогательный материал фасада_металл,Вспомогательный материал фасада_нет,Вспомогательный материал фасада_пластик,Вспомогательный материал фасада_стекло,Населенный пункт_артем,Населенный пункт_владивосток,...,Месяц_6,Месяц_7,Месяц_8,Месяц_9,Тип дома_другой,Тип дома_монолит,Тип дома_монолитный каркас,Тип дома_панель,Статус_сдан,Статус_строится
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [41]:
ds = pd.concat([ds.drop(object_ds.columns, axis=1), encoded_object_ds], axis=1)

ds.head()

,Таргет,Всероссийский застройщик,Инфраструктура: Рядом ВУЗ,Есть железная дорога 250м,Есть ЛЭП 100м,Есть магистральные дороги 100м,Есть АЗС 100м,Есть скверы 500м,Есть лес 500м,Есть водоем 500м,...,Месяц_6,Месяц_7,Месяц_8,Месяц_9,Тип дома_другой,Тип дома_монолит,Тип дома_монолитный каркас,Тип дома_панель,Статус_сдан,Статус_строится
0,95000.000000,0,1,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,105683.431953,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,110317.929760,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,115389.053254,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,132843.750000,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [45]:
ds.to_excel('src/dataset_processed.xlsx', index=False)